In [ ]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
import math
import numpy as np
from bayes_opt import BayesianOptimization

# iteration = 5
    

def preprocess(Data,max_norm=1.1,min_norm=0.1):
    #normalization
    Data = (Data - Data.min())/(Data.max() - Data.min())*(max_norm - min_norm) + min_norm
    # for i in range(len(Data.columns)):
    #     Data[Data.columns[i]] = (Data[Data.columns[i]] - Data[Data.columns[i]].mean())/Data[Data.columns[i]].std()
    #     print('standardized')
    #standardization
    #for i in range(len(Data.columns)):
    #    if Data[Data.columns[i]].std()*3 > (Data[Data.columns[i]].max()-Data[Data.columns[i]].min()):
    #        Data[Data.columns[i]] = (Data[Data.columns[i]] - Data[Data.columns[i]].mean())/Data[Data.columns[i]].std()
    #        print('standardized %d'%i)
    #    else:
    #        Data = (Data - Data.min())/(Data.max() - Data.min())*(max_norm - min_norm) + min_norm
    return Data

# Data = pd.read_csv(r'D:\Desktop\190423\youngjae_toy1.csv')
# # Data = np.log(Data)
# Data = preprocess(Data)


def target(x1,x2,x3):

    #### load data
        
    Data = pd.read_csv(r'D:\Desktop\one_signal.csv')
    Data = preprocess(Data)

    #     data_index = list(Data.columns)  # 나중에 그래프 그릴때 쓰기 위해 정의

    X= Data[Data.columns[:-1]]
    Y= Data[Data.columns[-1:]]

    x_data, x_test, y_data, y_test = train_test_split(X,Y, test_size=0.33, random_state=None)

    x_data = x_data.values.tolist()
    x_test = x_test.values.tolist()
    y_data = y_data.values.tolist()
    y_test = y_test.values.tolist()

    ########
    # predefined parameter
    #############################

    n = 3  # 뉴런 개수
    n_layer = 5 #네트워크 레이어 개수
    diag_act_f = tf.nn.sigmoid  #대각행렬 레이어 활성화 함수
    act_f = tf.nn.elu           #액티베이션 함수
    xavier_init = tf.keras.initializers.glorot_normal()
    he_init = tf.keras.initializers.he_normal()
    # he_init = tf.truncated_normal
    l_rate = 0.01 #러닝 레이트
    t_step = 50000 #트레이닝 스텝
    input_n = len(x_data[0]) #인풋 뉴런 개수

    ###############
    # model structure
    #################


    #Diagonal layer (compressed layer)

    diag_W = tf.constant([x1,x2,x3])
    diag_W1 = tf.diag(diag_W)
    diag_layerout = tf.matmul(x_data, diag_W1)
    test_diag_layerout = tf.matmul(x_test, diag_W1)

    #Neural network

    W1 = tf.Variable(he_init([input_n,n]))
    b1 = tf.Variable(he_init([n]))
    layer1out = act_f(tf.matmul(diag_layerout, W1) + b1)
    test_layer1out = act_f(tf.matmul(test_diag_layerout, W1) + b1)


    for i in range(n_layer-1):

        locals()['W{}'.format(i+2)] = tf.Variable(he_init([n,n]))
        locals()['b{}'.format(i+2)] = tf.Variable(he_init([n]))
        locals()['layer{}out'.format(i+2)] = act_f(tf.matmul(locals()['layer{}out'.format(i+1)], locals()['W{}'.format(i+2)]) + locals()['b{}'.format(i+2)])
        locals()['test_layer{}out'.format(i+2)] = act_f(tf.matmul(locals()['test_layer{}out'.format(i+1)], locals()['W{}'.format(i+2)]) + locals()['b{}'.format(i+2)])

    # model layer

    M_W = tf.Variable(he_init([n,1]))
    # M_b = tf.Variable(he_init([1]))
    M_layerout = tf.matmul(locals()['layer{}out'.format(n_layer)], M_W)
    test_M_layerout = tf.matmul(locals()['test_layer{}out'.format(n_layer)], M_W)

    ### training algorithm

    # loss_kl_train = tf.reduce_mean(tf.abs(y_data*tf.math.log(M_layerout/y_data)))
    # loss_kl_test = tf.reduce_mean(tf.abs(y_test*tf.math.log(test_M_layerout/y_test)))
    loss_train = tf.sqrt(tf.reduce_mean((tf.square(y_data - M_layerout))))  
    loss_test  = tf.sqrt(tf.reduce_mean((tf.square(y_test - test_M_layerout))))
    train = tf.train.AdamOptimizer(l_rate).minimize(loss_train)

    #################
    ###### train step
    #################

    #         train_rmse = []
    #         test_rmse = []

    sess = tf.Session()

    test_rmse = []
    for i in range(10):
        sess.run(tf.global_variables_initializer())

        # while math.isnan(sess.run(loss_kl_train)) == True:
        #     sess.run(tf.global_variables_initializer())

        for step in range(t_step):

            sess.run(train)


            ### save bset value

            if step == 0:
                best_value = sess.run(loss_test)
                over_num = 0

            else:   
                if best_value >= sess.run(loss_test):

                    over_num = 0
                    best_value = sess.run(loss_test)
                    # best_value1= sess.run(loss_train)
    #                     y_model = sess.run(test_M_layerout)
    #                     diagonal_matrix = sess.run(diag_W)

                if best_value < sess.run(loss_test):
                    over_num += 1

            if over_num == 100:
    #                     print('over fitting occurs!')
                break

                # if (step+1) % 10 == 0:

                #     print(step+1, 'steps    loss : ', sess.run(loss_train))

    #                 if (step+1) % 100 == 0:
    #                     train_rmse.append(sess.run(loss_train))
    #                     test_rmse.append(sess.run(loss_test))

            # if sess.run(loss_train) <= 1.0e-06:

            #     break
        test_rmse.append(best_value)

    return -min(test_rmse)

# def target(x1,x2,x3,x4,x5):
    
#     tf.constant([x1,x2,x3,x4,x5])

#     return -min(test_rmse)

bayes_optimizer = BayesianOptimization(target, {'x1': (0,1),'x2':(0,1),'x3':(0,1)},random_state=0)
bayes_optimizer.maximize(init_points=2, n_iter=30, acq='ei', xi=0.01)

|   iter    |  target   |    x1     |    x2     |    x3     |
-------------------------------------------------------------
Instructions for updating:
Colocations handled automatically by placer.
|  1        | -0.002276 |  0.5488   |  0.7152   |  0.6028   |
|  2        | -0.002422 |  0.5449   |  0.4237   |  0.6459   |
|  3        | -0.001494 |  0.9763   |  0.0335   |  0.04163  |
|  4        | -0.001922 |  0.01493  |  0.9619   |  0.7621   |
|  5        | -0.002689 |  0.08784  |  0.979    |  0.5038   |
|  6        | -0.00217  |  0.9209   |  0.6733   |  0.91     |
|  7        | -0.002093 |  0.2034   |  0.3607   |  0.9289   |
|  8        | -0.001484 |  0.9026   |  0.09608  |  0.9077   |
|  9        | -0.001944 |  0.9598   |  0.9639   |  0.3929   |
|  10       | -0.00479  |  0.008784 |  0.03267  |  0.01351  |
|  11       | -0.002647 |  0.9959   |  0.9817   |  0.02425  |
|  12       | -0.002049 |  0.04834  |  0.001104 |  0.9943   |
|  13       | -0.003477 |  0.007862 |  0.9922   |  0.004592 

In [14]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
import math
import numpy as np
from bayes_opt import BayesianOptimization

# iteration = 5
    
 
def preprocess(Data,max_norm=1.1,min_norm=0.1):
    #normalization
    Data = (Data - Data.min())/(Data.max() - Data.min())*(max_norm - min_norm) + min_norm
    # for i in range(len(Data.columns)):
    #     Data[Data.columns[i]] = (Data[Data.columns[i]] - Data[Data.columns[i]].mean())/Data[Data.columns[i]].std()
    #     print('standardized')
    #standardization
    #for i in range(len(Data.columns)):
    #    if Data[Data.columns[i]].std()*3 > (Data[Data.columns[i]].max()-Data[Data.columns[i]].min()):
    #        Data[Data.columns[i]] = (Data[Data.columns[i]] - Data[Data.columns[i]].mean())/Data[Data.columns[i]].std()
    #        print('standardized %d'%i)
    #    else:
    #        Data = (Data - Data.min())/(Data.max() - Data.min())*(max_norm - min_norm) + min_norm
    return Data

# Data = pd.read_csv(r'D:\Desktop\190423\youngjae_toy1.csv')
# # Data = np.log(Data)
# Data = preprocess(Data)


#### load data

Data = pd.read_csv(r'D:\Desktop\case1_Flame.csv')
Data = preprocess(Data)

#     data_index = list(Data.columns)  # 나중에 그래프 그릴때 쓰기 위해 정의

X= Data[Data.columns[:-1]]
Y= Data[Data.columns[-1:]]

x_data, x_test, y_data, y_test = train_test_split(X,Y, test_size=0.33, random_state=None)

x_data = x_data.values.tolist()
x_test = x_test.values.tolist()
y_data = y_data.values.tolist()
y_test = y_test.values.tolist()

########
# predefined parameter
#############################

n = 4  # 뉴런 개수
n_layer = 3 #네트워크 레이어 개수
diag_act_f = tf.nn.sigmoid  #대각행렬 레이어 활성화 함수
act_f = tf.nn.elu           #액티베이션 함수
xavier_init = tf.keras.initializers.glorot_normal()
he_init = tf.keras.initializers.he_normal()
# he_init = tf.truncated_normal
l_rate = 0.01 #러닝 레이트
t_step = 50000 #트레이닝 스텝
input_n = len(x_data[0]) #인풋 뉴런 개수

###############
# model structure
#################


#Diagonal layer (compressed layer)

diag_W = tf.Variable([ 0.8053, 0.9967,  0.4515, 0.9983])
# diag_W = tf.clip_by_value(diag_W, 0.0,1.0)
diag_W1 = tf.diag(diag_W)
diag_layerout = tf.matmul(x_data, diag_W1)
test_diag_layerout = tf.matmul(x_test, diag_W1)

#Neural network

W1 = tf.Variable(he_init([input_n,n]))
b1 = tf.Variable(he_init([n]))
layer1out = act_f(tf.matmul(diag_layerout, W1) + b1)
test_layer1out = act_f(tf.matmul(test_diag_layerout, W1) + b1)


for i in range(n_layer-1):

    locals()['W{}'.format(i+2)] = tf.Variable(he_init([n,n]))
    locals()['b{}'.format(i+2)] = tf.Variable(he_init([n]))
    locals()['layer{}out'.format(i+2)] = act_f(tf.matmul(locals()['layer{}out'.format(i+1)], locals()['W{}'.format(i+2)]) + locals()['b{}'.format(i+2)])
    locals()['test_layer{}out'.format(i+2)] = act_f(tf.matmul(locals()['test_layer{}out'.format(i+1)], locals()['W{}'.format(i+2)]) + locals()['b{}'.format(i+2)])

# model layer

M_W = tf.Variable(he_init([n,1]))
# M_b = tf.Variable(he_init([1]))
M_layerout = tf.matmul(locals()['layer{}out'.format(n_layer)], M_W)
test_M_layerout = tf.matmul(locals()['test_layer{}out'.format(n_layer)], M_W)

### training algorithm

# loss_kl_train = tf.reduce_mean(tf.abs(y_data*tf.math.log(M_layerout/y_data)))
# loss_kl_test = tf.reduce_mean(tf.abs(y_test*tf.math.log(test_M_layerout/y_test)))
loss_train = tf.sqrt(tf.reduce_mean((tf.square(y_data - M_layerout))))  
loss_test  = tf.sqrt(tf.reduce_mean((tf.square(y_test - test_M_layerout))))
train = tf.train.AdamOptimizer(l_rate).minimize(loss_train)

#################
###### train step
#################

#         train_rmse = []
#         test_rmse = []

sess = tf.Session()

test_rmse = []
for i in range(10):
    sess.run(tf.global_variables_initializer())

    # while math.isnan(sess.run(loss_kl_train)) == True:
    #     sess.run(tf.global_variables_initializer())

    for step in range(t_step):

        sess.run(train)


        ### save bset value

        if step == 0:
            best_value = sess.run(loss_test)
            over_num = 0

        else:   
            if best_value >= sess.run(loss_test):

                over_num = 0
                best_value = sess.run(loss_test)
                diagonal_matrix = sess.run(diag_W)
                # best_value1= sess.run(loss_train)
#                     y_model = sess.run(test_M_layerout)
#                     diagonal_matrix = sess.run(diag_W)

            if best_value < sess.run(loss_test):
                over_num += 1

        if over_num == 100:
#                     print('over fitting occurs!')
            break

            # if (step+1) % 10 == 0:

            #     print(step+1, 'steps    loss : ', sess.run(loss_train))

#                 if (step+1) % 100 == 0:
#                     train_rmse.append(sess.run(loss_train))
#                     test_rmse.append(sess.run(loss_test))

        # if sess.run(loss_train) <= 1.0e-06:

        #     break
    print('training reslut %d :   ' %i,best_value)
    print(diagonal_matrix)




training reslut 0 :    0.027470049
[4.8430185 1.8330048 1.901322  5.4408255]
training reslut 1 :    0.026021564
[2.1470091  2.4607403  0.56423783 3.0229812 ]
training reslut 2 :    0.022785846
[1.7917668 3.8629305 1.1018832 6.4867516]
training reslut 3 :    0.04370553
[0.1831401  0.96483344 0.53823096 1.9802734 ]
training reslut 4 :    0.02859387
[2.0024457  0.93272144 0.89835435 3.0513265 ]
training reslut 5 :    0.037512377
[ 1.428033    1.1669707  -0.54871017  1.9919406 ]
training reslut 6 :    0.02640528
[2.013637  1.4738048 1.0057619 3.4788268]
training reslut 7 :    0.031106029
[2.1208034 1.589889  0.5524596 1.1625216]


KeyboardInterrupt: 

In [50]:
tf.constant([1,2,3])

<tf.Tensor 'Const_52:0' shape=(3,) dtype=int32>

In [51]:
def target(x,y):
    return np.exp(-(x-3)**2) + np.exp(-(3*y-2)**2) + 1/(x**2+1)

bayes_optimizer = BayesianOptimization(target, {'x':(-2,6),'y':(-2,6)}, random_state=0)
bayes_optimizer.maximize(init_points=2, n_iter=14, acq='ei', xi=0.01)

|   iter    |  target   |     x     |     y     |
-------------------------------------------------
|  1        |  0.8386   |  2.391    |  3.722    |
|  2        |  1.08     |  2.822    |  2.359    |
|  3        |  0.02715  |  6.0      | -2.0      |
|  4        |  0.5412   | -0.9906   |  0.06011  |
|  5        |  0.02715  |  6.0      |  4.0      |
|  6        |  0.2      | -2.0      |  6.0      |
|  7        |  0.6005   |  1.983    |  1.259    |
|  8        |  0.2      | -2.0      | -2.0      |
|  9        |  0.4923   |  3.917    |  2.08     |
|  10       |  0.2      | -2.0      |  2.475    |
|  11       |  0.6258   |  2.094    |  2.659    |
|  12       |  1.088    |  3.085    |  6.0      |
|  13       |  0.4249   |  1.621    |  6.0      |
|  14       |  0.7525   |  3.619    |  4.894    |
|  15       |  0.4429   |  1.705    | -2.0      |
|  16       |  0.1361   |  4.551    |  6.0      |


In [16]:
a = []
print(len(a))
a= a*100
print(len(a))

0
0


In [17]:
import pickle

In [24]:
a = ['a','b','c']
b = {'a':1,'b':2,'c':3}
with open('a.txt','wb') as f:
    pickle.dump(a, f)
    pickle.dump(b, f)
    

In [27]:
with open('a.txt','rb') as f:
    data1 = pickle.load(f)
    data2 = pickle.load(f)

data1

In [28]:
data1

['a', 'b', 'c']

In [29]:
data2

{'a': 1, 'b': 2, 'c': 3}

In [46]:
a = ['x1','x2','x3']
b = {}
for i in range(len(a)):
    b['%s'%a[i]] = 0
b

{'x1': 0, 'x2': 0, 'x3': 0}

In [59]:
def selection_worklist(parameter_list, select_order):

    for i in range(len(select_order)):
        parameter_list.remove(select_order[i])

    work_dictionary = {}
    
    for i in range(len(parameter_list)):
        work_dictionary['%s'%parameter_list[i]] = 0

    return work_dictionary

a = ['x1','x2','x3','x4','x5']
b = ['x3','x5']
c = selection_worklist(a,b)

In [104]:
def selection_worklist(parameter_list, select_order):

    for i in select_order:
        parameter_list.remove(i)

    work_dictionary = {}
    
    for i in parameter_list:
        work_dictionary[i] = 0

    return work_dictionary

a = ['x1','x2','x3','x4','x5']
b = ['x3','x5']
c = selection_worklist(a,b)
c

{'x1': 0, 'x2': 0, 'x4': 0}

In [110]:
a = {'x1':0,'x2':1,'x3':2}
min(a.keys())

'x1'

In [57]:
c = list(b.keys())
for i in range(len(b)):
    print(b[c[i]])

0
0
0


In [61]:
for i in d.keys():
    c[i] = d[i]
print(c)

{'x2': 2, 'x4': 0, 'x1': 1, 'x3': 3}


In [68]:
load_directory = r'D:\Desktop\youngjae_toy1.csv'
Data = pd.read_csv(load_directory)
Data

,x1,x2,x3,x4,x5,y
0,0.078159,0.279569,0.726339,0.383194,0.876992,0.059308
1,0.858761,0.926694,0.040198,0.000065,0.093738,0.405621
2,0.355820,0.596506,0.625106,0.244264,0.520351,0.117702
3,0.602916,0.776477,0.573885,0.189006,0.280197,0.184696
4,0.579876,0.761496,0.291364,0.024735,0.461884,0.204433
5,0.176440,0.420048,0.638396,0.260177,0.024173,0.066339
6,0.713760,0.844843,0.501184,0.125890,0.566459,0.214129
7,0.741028,0.860830,0.523855,0.143758,0.771799,0.222707
8,0.170338,0.412720,0.551888,0.168095,0.498682,0.052986
9,0.784993,0.885998,0.460329,0.097545,0.528759,0.236600


In [69]:
a = {}
a['x2'] = 1
a

{'x2': 1}

In [100]:
a = {'x1':0,'x2':0,'x3':0}
b = pd.DataFrame(a.items(), columns=['selected parameter','RMSE'])
b

,selected parameter,RMSE
0,x2,0
1,x1,0
2,x3,0


In [91]:
data = pd.DataFrame(Data, columns=a.keys())
data

,x2,x1,x3
0,0.279569,0.078159,0.726339
1,0.926694,0.858761,0.040198
2,0.596506,0.355820,0.625106
3,0.776477,0.602916,0.573885
4,0.761496,0.579876,0.291364
5,0.420048,0.176440,0.638396
6,0.844843,0.713760,0.501184
7,0.860830,0.741028,0.523855
8,0.412720,0.170338,0.551888
9,0.885998,0.784993,0.460329


In [86]:
data['x4'] = Data['x4']
data['y'] = Data['y']
data

,x2,x1,x3,x4,y
0,0.279569,0.078159,0.726339,0.383194,0.059308
1,0.926694,0.858761,0.040198,0.000065,0.405621
2,0.596506,0.355820,0.625106,0.244264,0.117702
3,0.776477,0.602916,0.573885,0.189006,0.184696
4,0.761496,0.579876,0.291364,0.024735,0.204433
5,0.420048,0.176440,0.638396,0.260177,0.066339
6,0.844843,0.713760,0.501184,0.125890,0.214129
7,0.860830,0.741028,0.523855,0.143758,0.222707
8,0.412720,0.170338,0.551888,0.168095,0.052986
9,0.885998,0.784993,0.460329,0.097545,0.236600


In [92]:
data[Data.columns[-1]] = Data[Data.columns[-1]]
data

,x2,x1,x3,y
0,0.279569,0.078159,0.726339,0.059308
1,0.926694,0.858761,0.040198,0.405621
2,0.596506,0.355820,0.625106,0.117702
3,0.776477,0.602916,0.573885,0.184696
4,0.761496,0.579876,0.291364,0.204433
5,0.420048,0.176440,0.638396,0.066339
6,0.844843,0.713760,0.501184,0.214129
7,0.860830,0.741028,0.523855,0.222707
8,0.412720,0.170338,0.551888,0.052986
9,0.885998,0.784993,0.460329,0.236600


In [113]:
device=[]
for i in range(2):
    device.append('/gpu:%d'%i)
device

['/gpu:0', '/gpu:1']

In [97]:
a = [1,2,3,4,5,6]

def delete_error(a):
    if len(a) < 3:
        mean_error = sum(a)/len(a)
    else:
        a.remove(max(a))
        a.remove(min(a))
        mean_error = sum(a)/len(a)
    
    return mean_error

delete_error(a)
        

3.5

In [114]:
a = {'':0}
a

{'': 0}

In [129]:
Data = pd.read_csv(r'D:\Desktop\190305\csvfile\new\새 폴더 (3)\0d_mean_orig_channel.csv')
Data.columns[-1]

'PHI-Pr_t_new'

In [136]:
y_series = Data[Data.columns[-1]]
y_series
data = pd.DataFrame(y_series)
data

,PHI-Pr_t_new
0,1.077835
1,1.030240
2,1.026915
3,1.076085
4,1.023545
5,1.025885
6,1.012440
7,1.071695
8,1.015165
9,1.016510


In [137]:
data['y1'] = y_series
data['y2'] = y_series

,PHI-Pr_t_new,y1
0,1.077835,1.077835
1,1.030240,1.030240
2,1.026915,1.026915
3,1.076085,1.076085
4,1.023545,1.023545
5,1.025885,1.025885
6,1.012440,1.012440
7,1.071695,1.071695
8,1.015165,1.015165
9,1.016510,1.016510


In [140]:
num_q_value = 5

y_list = ['y%d'%i for i in range(num_q_value-1)]
y_list

['y0', 'y1', 'y2', 'y3']

In [141]:
data = pd.DataFrame(y_series)
for i in y_list:
    data[i] = y_series
data

,PHI-Pr_t_new,y0,y1,y2,y3
0,1.077835,1.077835,1.077835,1.077835,1.077835
1,1.030240,1.030240,1.030240,1.030240,1.030240
2,1.026915,1.026915,1.026915,1.026915,1.026915
3,1.076085,1.076085,1.076085,1.076085,1.076085
4,1.023545,1.023545,1.023545,1.023545,1.023545
5,1.025885,1.025885,1.025885,1.025885,1.025885
6,1.012440,1.012440,1.012440,1.012440,1.012440
7,1.071695,1.071695,1.071695,1.071695,1.071695
8,1.015165,1.015165,1.015165,1.015165,1.015165
9,1.016510,1.016510,1.016510,1.016510,1.016510


In [142]:
a= 1
b=2
c=3
q_value = [a,b,c]
q_value

[1, 2, 3]

In [152]:
a = 1,2,3
a.index(max(a))

2

In [146]:
a = {'%d'%i:10*i for i in range(3)}
a

{'0': 0, '1': 10, '2': 20}

In [150]:
min(a)

str